<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Supp_Fig_6/common_macaque_infections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plot counts of virus genera commonly infecting macaques
Reference: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7150127/

In [ ]:
!pip install -q anndata
import anndata
import numpy as np
import pandas as pd
import json
import os
import glob
import matplotlib.pyplot as plt
from matplotlib import colors
%config InlineBackend.figure_format='retina'

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

___
# Load data
The count matrix was generated as shown [here](https://github.com/pachterlab/LSCHWCP_2023/tree/main/Notebooks/align_macaque_PBMC_data/7_virus_host_captured_dlist_cdna_dna).

In [ ]:
# Download count matrix from Caltech Data
!wget https://data.caltech.edu/records/sh33z-hrx98/files/virus_host-captured_dlist_cdna_dna.h5ad?download=1
!mv virus_host-captured_dlist_cdna_dna.h5ad?download=1 virus_host-captured_dlist_cdna_dna.h5ad

In [ ]:
palmdb_adata = anndata.read("virus_host-captured_dlist_cdna_dna_also-in-host.h5ad")
palmdb_adata

In [ ]:
# Load virus ID to taxonomy mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
id2tax = pd.read_csv("ID_to_taxonomy_mapping.csv")

____

In [ ]:
## -> family
# -> genus that had no counts
genera_to_test = [
    ### Nonenveloped RNA-Containing Viruses

    ## Reoviridae
    "Orthoreovirus",
    "Rotavirus",

    ## Picornaviridae
    "Cardiovirus",
    "Enterovirus",
    "Hepatovirus",
    "Sapelovirus",

    # Caliciviridae
    "Norovirus",
    "Recovirus",
    "Vesivirus",

    ## Coronaviridae
    "Alphacoronavirus",
    "Betacoronavirus",
    "Deltacoronavirus",
    "Gammacoronavirus",

    ## Hepeviridae
    'Orthohepevirus',
    'Piscihepevirus',

    ### Enveloped RNA-Containing Viruses

    ## Arenaviridae
    "Arenavirus",
    "Mammarenavirus",

    ## Flaviviridae
    "Hepacivirus",
    "Flavivirus",
    "Pestivirus",

    # Togaviridae
    "Alphavirus",
    "Rubivirus",

    # Arteriviridae
    "arterivirus",

    # Paramyxoviridae
    "Morbillivirus",
    "Metapneumovirus",
    "Respirovirus",

    ## Orthomyxoviridae
    'Alphainfluenzavirus',
    'Betainfluenzavirus',
    'Deltainfluenzavirus',
    'Gammainfluenzavirus',

    # Filoviridae
    "Ebolavirus",
    "Marburgvirus",

    # Rhabdoviridae
    "Lyssavirus",
]

In [ ]:
def rand_jitter(arr, jitter_size=0.02):
    """
    Add random jitter to array to avoid overplotting.
    """
    return np.random.uniform(low=-jitter_size, high=jitter_size, size=(len(arr),))

In [ ]:
# Define order of animals
animal_ids = [
    'NHP1',
    'NHPC1',
    'NHP2',
    'NHP08',
    'NHP09',
    'NHP10',
    'NHP11',
    'NHP12',
    'NHP13',
    'NHP14',
    'NHP15',
    'NHP16',
    'NHP17',
    'NHP18',
    'NHP19',
    'NHP20',
    'NHP22',
    'NHP23',
    'NHP24',
]

In [ ]:
len(animal_ids)

In [ ]:
# # Filtering options:
# palmdb_adata = palmdb_adata[palmdb_adata.obs["celltype"].notnull(), palmdb_adata.var["v_type"] == "macaca_only"].copy()
# palmdb_adata = palmdb_adata[palmdb_adata.obs["celltype"].notnull(), palmdb_adata.var["v_type"] != "below_threshold"].copy()
# palmdb_adata = palmdb_adata[palmdb_adata.obs["celltype"].notnull(), :].copy()

In [ ]:
%%time
fig, axs = plt.subplots(figsize=(30,30), nrows=int(len(genera_to_test)/3), ncols=3, sharex=True)
fontsize = 14
width = 0.7
NUM_COLORS = 20

x = np.arange(len(animal_ids))

col_idx = 0
row_idx = 0
for genus in genera_to_test:
    target_species = id2tax[id2tax["genus"].str.contains(genus)]["species"].unique()

    color_idx = 0
    second_hatch_round=False
    hatch = None
    low_counts = True
    bottoms = np.zeros(len(animal_ids))
    for species in target_species:
        target_ids = id2tax[(id2tax["genus"].str.contains(genus)) & (id2tax["species"] == species)]["rep_ID"].unique()

        # Check if virus IDs are in adata
        if len(palmdb_adata.var[palmdb_adata.var.index.isin(target_ids)]) > 0:
            counts = []
            for i, animal in enumerate(animal_ids):
                num_cells = len(
                    palmdb_adata[
                        palmdb_adata.obs["donor_animal"] == animal,
                        palmdb_adata.var.index.isin(target_ids)
                    ]
                )

                # Number of virus+ cells
                num_cells_virus = palmdb_adata[
                            palmdb_adata.obs["donor_animal"] == animal,
                            palmdb_adata.var.index.isin(target_ids)
                        ].X.sum()

                # Count per 10k cells
                norm_count = (num_cells_virus / num_cells) * 10000

                counts.append(norm_count)
        else:
            counts = np.zeros(len(animal_ids))

        if species == ".":
            label = "Undefined"
        else:
            label = species

        ax = axs[row_idx, col_idx]
        ax.bar(x, counts, bottom=bottoms, width=width, label=label, hatch=hatch, color=plt.cm.tab20(color_idx), edgecolor="black")

        if color_idx == NUM_COLORS-1 and not second_hatch_round:
            hatch = "/"
            second_hatch_round = True
            color_idx = 0
        elif color_idx == NUM_COLORS-1 and second_hatch_round:
            hatch = "\\"
            color_idx = 0

        bottoms = bottoms + np.array(counts)
        color_idx += 1

        if np.max(bottoms) >= 5:
            low_counts = False

    # ax.text(x[i], norm_count, "{:,}".format(num_cells), ha="center", va="bottom", fontsize=fontsize-7)

    ax.set_title(genus, fontsize=fontsize+2)
    if col_idx == 0:
        if row_idx == 4:
            ax.set_ylabel("# positive cells per 10k cells", fontsize=fontsize+5)

    if col_idx == 1:
        if row_idx == len(genera_to_test) / 3 - 1:
            ax.set_xlabel("Animal ID", fontsize=fontsize+5)
    ax.tick_params(axis="both", labelsize=fontsize)
    ax.set_xticks(x)

    if low_counts:
        ax.set_ylim(bottom=0, top=5)
    else:
        ax.set_ylim(bottom=0, top=np.max(bottoms)+np.max(bottoms)*0.1)

    if len(target_species)>8 and len(target_species)<15:
        # ax.legend(fontsize=fontsize-2, loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
        ax.legend(fontsize=fontsize-5, loc='upper right', ncol=2)
    elif len(target_species)>=15 and len(target_species)<50:
        # ax.legend(fontsize=fontsize-2, loc='center left', bbox_to_anchor=(1, 0.5), ncol=3)
        ax.legend(fontsize=fontsize-8, loc='upper right', ncol=3)
    elif len(target_species)>=50:
        # ax.legend(fontsize=fontsize-2, loc='center left', bbox_to_anchor=(1, 0.5), ncol=5)
        ax.legend(fontsize=fontsize-9, loc='upper right', ncol=5)
    else:
        # ax.legend(fontsize=fontsize-2, loc='center left', bbox_to_anchor=(1, 0.5))
        ax.legend(fontsize=fontsize-5, loc='upper right')

    ax.grid(True, which="both", color="lightgray")
    ax.set_axisbelow(True)

    ax.margins(0.01)

    row_idx += 1
    if row_idx == len(genera_to_test) / 3:
        ax.set_xticklabels(animal_ids, rotation=45, ha="right", fontsize=fontsize)
        row_idx = 0
        col_idx += 1

plt.subplots_adjust(hspace=0.2, wspace=0.06)

# plt.tight_layout()

plt.savefig(
    "common_macaque_infection_counts.png",
    bbox_inches="tight",
    dpi=300
)

fig.show()